# RNN Encoder-decoder non-vectorized

Use [fastai book chap 12](https://github.com/fastai/fastbook/blob/master/12_nlp_dive.ipynb) human numbers data to train translator from english like "two hundred seven" to sequence of digits like "207". Data looks like:

```
one 
two 
three 
...
two hundred seven 
two hundred eight 
...
```

In [1]:
from fastai2.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)
path

Path('/Users/parrt/.fastai/data/human_numbers')

## Support

In [2]:
import codecs
import os
import re
import string
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
#from torch.nn.functional import softmax
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

dtype = torch.float
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
def get_text(filename:str):
    """
    Load and return the text of a text file, assuming latin-1 encoding as that
    is what the BBC corpus uses.  Use codecs.open() function not open().
    """
    f = codecs.open(filename, encoding='latin-1', mode='r')
    s = f.read()
    f.close()
    return s

In [4]:
def getvocab(strings):
    letters = [list(l) for l in strings]
    vocab = set([c for cl in letters for c in cl])
    vocab = sorted(list(vocab))
    ctoi = {c:i for i, c in enumerate(vocab)}
    return vocab, ctoi

In [5]:
def softmax(y):
    expy = torch.exp(y)
    if len(y.shape)==1: # 1D case can't use axis arg
        return expy / torch.sum(expy)
    return expy / torch.sum(expy, axis=1).reshape(-1,1)

In [6]:
def get_max_len(X):
    max_len = 0
    for x in X:
        max_len = max(max_len, len(x))
    return max_len

## Load

In [7]:
text = get_text(path/'train.txt').strip()
print(text[:28])
lines = text.lower().split('\n')
print(lines[:5])

one 
two 
three 
four 
five 
['one ', 'two ', 'three ', 'four ', 'five ']


In [8]:
lines = lines[0:1000]

In [9]:
# get unique vocab but don't sort; keep order so 'one'=1 etc...
# use '#' to indicate padded (unused) char for embedding purposes
v = set('#')
X_vocab = ['#']
for t in text.split():
    if t not in v:
        X_vocab.append(t)
        v.add(t)
X_vocab[:10]

['#', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

In [10]:
X_tokens = [line.strip().split(' ') for line in lines]
X_tokens[18:23]

[['nineteen'],
 ['twenty'],
 ['twenty', 'one'],
 ['twenty', 'two'],
 ['twenty', 'three']]

In [11]:
n = len(X_tokens)
X_vocab = {w:i for i,w in enumerate(X_vocab)}
X_idx = {i:w for i,w in enumerate(X_vocab)}
X_vocab['one'], X_vocab['eleven'], X_idx[1], X_idx[11]

(1, 11, 'one', 'eleven')

In [12]:
# numericalize but don't pad
X = []
for i in range(len(X_tokens)):
    x = X_tokens[i]
    X.append( [X_vocab[w] for w in x] )
X[15:30]

[[16],
 [17],
 [18],
 [19],
 [20],
 [20, 1],
 [20, 2],
 [20, 3],
 [20, 4],
 [20, 5],
 [20, 6],
 [20, 7],
 [20, 8],
 [20, 9],
 [21]]

## Translation

### Define y sequence of digits

Let's use Y as list of lists like X; targets like `'one' -> '1'`, `['twenty', 'three'] -> ['2','3']`, etc...

Use '<' for start of sequence and '>' for end. So sequence `ab` is stored `<ab>`.



In [13]:
Y_vocab = {d:i for i,d in enumerate("0123456789<>")}
Y_idx = {i:w for i,w in enumerate("0123456789<>")}
Y_vocab

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 '<': 10,
 '>': 11}

In [14]:
Ystr = [f"<{i+1}>" for i in range(0,len(X))]
Y_max_len = get_max_len(Ystr)
Ystr[:11]

['<1>', '<2>', '<3>', '<4>', '<5>', '<6>', '<7>', '<8>', '<9>', '<10>', '<11>']

In [15]:
Y = []
for i in range(0,len(X)):
    y = Ystr[i]
#    pad = Y_max_len - len(y)
    Y.append([Y_vocab[d] for d in y])
#Y = torch.tensor(Y)
Y[19:25]

[[10, 2, 0, 11],
 [10, 2, 1, 11],
 [10, 2, 2, 11],
 [10, 2, 3, 11],
 [10, 2, 4, 11],
 [10, 2, 5, 11]]

In [16]:
embed_sz = 20
y_embed_sz = 5
nhidden = 256
nclasses = len(Y_vocab) # char output vocab

print(f"{n:,d} training records, {nclasses} target classes, state is {nhidden}-vector")

1,000 training records, 12 target classes, state is 256-vector


### Split out validation set

Not sure this will generalize but...

In [17]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

### Train

In [18]:
def forward(x, y):
    # ENCODER
    h = torch.zeros(nhidden, 1, device=device, dtype=torch.float64, requires_grad=False)  # reset hidden state at start of record
    for t in range(len(x)):
        embedding_step_t = Ex[:,x[t]]
        embedding_step_t = embedding_step_t.reshape(embed_sz,1)
        h = W @ h + U @ embedding_step_t + bx
        h = torch.tanh(h)

    # DECODER
    loss = 0.0
    correct = 0
#     print("DECODE", y)
    for t in range(len(y)-1): # don't predict next char at final '>'
        embedding_step_t = Ey[:,y[t]]
        embedding_step_t = embedding_step_t.reshape(y_embed_sz,1)
        h = W2 @ h + U2 @ embedding_step_t + by
        h = torch.tanh(h)
        o = V @ h + bo
        o = o.reshape(1,nclasses)
        # From y we want to predict y[1:]. at y[t], predict y[t+1]
        loss += F.cross_entropy(o, torch.tensor([y[t+1]], device=device))

        p = softmax(o)
        correct += torch.argmax(p[0]).item()==y[t+1]
#         epoch_training_accur += correct
    return loss, int(correct)

In [19]:
#%%time 
#torch.manual_seed(0) # SET SEED FOR TESTING
Ex = torch.randn(embed_sz,     len(X_vocab),  device=device, dtype=torch.float64, requires_grad=True) # embedding
W = torch.eye(nhidden,         nhidden,       device=device, dtype=torch.float64, requires_grad=True)
U = torch.randn(nhidden,       embed_sz,      device=device, dtype=torch.float64, requires_grad=True) # input converter
bx = torch.zeros(nhidden,      1,             device=device, dtype=torch.float64, requires_grad=True)
by = torch.zeros(nhidden,      1,             device=device, dtype=torch.float64, requires_grad=True)
bo = torch.zeros(nclasses,     1,             device=device, dtype=torch.float64, requires_grad=True)

Ey = torch.randn(y_embed_sz,   len(Y_vocab),  device=device, dtype=torch.float64, requires_grad=True) # embedding
W2 = torch.eye(nhidden,        nhidden,       device=device, dtype=torch.float64, requires_grad=True)
#C = torch.randn(nhidden,       nhidden,       device=device, dtype=torch.float64, requires_grad=True) # input converter
U2 = torch.randn(nhidden,      y_embed_sz,    device=device, dtype=torch.float64, requires_grad=True) # input converter
V = torch.randn(nclasses,      nhidden,       device=device, dtype=torch.float64, requires_grad=True)

optimizer = torch.optim.Adam([Ex,W,U,Ey,W2,U2,V,bx,by,bo], lr=0.001, weight_decay=0.0)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 
                                              mode='triangular2',
                                              step_size_up=4,
                                              base_lr=0.0005, max_lr=0.005,
                                              cycle_momentum=False)

torch.autograd.set_detect_anomaly(True)

history = []
epochs = 35
for epoch in range(1, epochs+1):
    epoch_training_loss = 0.0
    epoch_training_accur = 0.0
    total_compares = 0
    for i in range(n):
        x = X[i]
        y = Y[i]
        loss, correct = forward(x, y)
        epoch_training_accur += correct
        epoch_training_loss += loss.detach().item()
        total_compares += len(y) - 1

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    scheduler.step()
    epoch_training_loss /= n
    epoch_training_accur /= total_compares
    print(f"Epoch {epoch:3d} training loss {epoch_training_loss:7.4f} accur {epoch_training_accur:7.4f}   LR {scheduler.get_last_lr()[0]:7.6f}")

Epoch   1 training loss 19.0493 accur  0.5371   LR 0.001625
Epoch   2 training loss 11.3777 accur  0.6930   LR 0.002750
Epoch   3 training loss 10.3474 accur  0.6943   LR 0.003875
Epoch   4 training loss 10.5952 accur  0.6432   LR 0.005000
Epoch   5 training loss  9.2944 accur  0.6432   LR 0.003875
Epoch   6 training loss  7.8974 accur  0.6278   LR 0.002750
Epoch   7 training loss  6.6700 accur  0.6455   LR 0.001625
Epoch   8 training loss  5.7310 accur  0.6273   LR 0.000500
Epoch   9 training loss  5.1907 accur  0.5993   LR 0.001063
Epoch  10 training loss  4.4192 accur  0.6648   LR 0.001625
Epoch  11 training loss  4.6658 accur  0.6784   LR 0.002188
Epoch  12 training loss  4.9271 accur  0.6851   LR 0.002750
Epoch  13 training loss  5.0474 accur  0.6902   LR 0.002188
Epoch  14 training loss  4.4758 accur  0.6930   LR 0.001625
Epoch  15 training loss  4.4699 accur  0.6915   LR 0.001063
Epoch  16 training loss  3.8225 accur  0.7038   LR 0.000500
Epoch  17 training loss  3.2103 accur  0

In [20]:
def sample(x):
    n = len(x)
    output = []
    with torch.no_grad():
        # ENCODER
        h = torch.zeros(nhidden, 1, device=device, dtype=torch.float64, requires_grad=False)  # reset hidden state at start of record
        for t in range(len(x)):
            embedding_step_t = Ex[:,x[t]]
            embedding_step_t = embedding_step_t.reshape(embed_sz,1)
            h = W @ h + U @ embedding_step_t + bx
            h = torch.tanh(h)

        # DECODER
        y = [Y_vocab['<']] # begin with "start of sequence" char
        loss = 0.0
        correct = 0
        while y!=Y_vocab['>']:
            embedding_step_t = Ey[:,y]
            embedding_step_t = embedding_step_t.reshape(y_embed_sz,1)
            h = W2 @ h + U2 @ embedding_step_t + by
            h = torch.tanh(h)
            o = V @ h + bo
            o = o.reshape(1,nclasses)
            p = softmax(o[0])
            y = torch.argmax(p).item()
            if y!=Y_vocab['>']:
                output.append(Y_idx[y])
    return output

In [21]:
x = [X_vocab[w] for w in "one hundred ten".split()]
print(x, [X_idx[n] for n in x])

[1, 28, 10] ['one', 'hundred', 'ten']


In [22]:
x = [X_vocab[w] for w in "one".split()]
output = sample(x)
print([X_idx[n] for n in x],'=>', output)

['one'] => ['1']


In [23]:
x = [X_vocab[w] for w in "one hundred".split()]
output = sample(x)
print([X_idx[n] for n in x],'=>', output)

['one', 'hundred'] => ['1', '0', '0']


In [24]:
x = [X_vocab[w] for w in "one hundred ten".split()]
output = sample(x)
print([X_idx[n] for n in x],'=>', output)

['one', 'hundred', 'ten'] => ['1', '1', '4']


In [25]:
x = [X_vocab[w] for w in "one hundred thirty two".split()]
output = sample(x)
print([X_idx[n] for n in x],'=>', output)

['one', 'hundred', 'thirty', 'two'] => ['1', '3', '2']


In [26]:
x = [X_vocab[w] for w in "eleven".split()]
output = sample(x)
print([X_idx[n] for n in x],'=>', output)

['eleven'] => ['1', '1']


In [27]:
x = [X_vocab[w] for w in "ninety nine".split()]
output = sample(x)
print([X_idx[n] for n in x],'=>', output)

['ninety', 'nine'] => ['9', '9']


In [28]:
x = [X_vocab[w] for w in "fifty three".split()]
output = sample(x)
print([X_idx[n] for n in x],'=>', output)

['fifty', 'three'] => ['5', '5', '5']
